In [1]:
import pandas as pd
import seaborn as sns

In [3]:
url = 'https://raw.githubusercontent.com/Lesadt/Practice/main/NORSMAN/Cleaned_NORSMAN'

df = pd.read_csv(url, sep=';', encoding = 'iso-8859-1')
df

HTTPError: HTTP Error 404: Not Found

In [ ]:
df.info()

Detected Problems in the dataset:
1. Last row is empty: Drop Row
2. Club column is not relevant to the project
3. First Name and Surname can be put together as 1 column
4. Columns with time objects needs to be changed to datetime format
    4.1. Wrong format in the 'Time T1' and 'Time T2' columns
5. Missing Countries: Ignore for now
6. Missing data in column 'Run Distance'
7. Drop year 2015 for having diferent swimming distance
8. Inicialy drop 2005 for having to many errors (save for later)

In [ ]:
#Drop last row
df = df.drop(index = 4076)
df['Name'] = df['First name'] + ' ' + df['Surname']
df.insert(0, 'Name', df.pop('Name'))
df = df.drop(['Club','First name', 'Surname'] , axis = 'columns')
df.head()

In [ ]:
#Deal with format errors to be able to convert Times columns to time format.
#for i in range(len(df)):
#    if str(df['Cycle time'][i]).count(':') != 2:
#        if pd.notna(df['Cycle time'][i]) == True:
#            print(i)
df['Cycle time'][232] = '8:55:34'
fix_col_T1_T2 = ['Time T1', 'Time T2']
for col in fix_col_T1_T2:
    for i in range(len(df)):
        if str(df[col][i]).count(':') != 2:
            if pd.notna(df[col][i]) == True:
                df[col][i] = '0:' + df[col][i]

In [ ]:
#Convert columns to 
time_col = ['Swim time', 'Time T1', 'Cycle time', 'Time T2', 'Run time', 'Total time']
for col in time_col:
    df[col] = pd.to_timedelta(df[col])
df.info()

In [ ]:
one_hour = pd.to_timedelta('0 days 01:00:00')
for i in range(len(df)):
    if df['Time T1'][i] > one_hour:
                df['Time T1'][i] = (df['Time T1'][i]/60).round(freq='S')
for i in range(len(df)):
    if df['Time T2'][i]> one_hour:
                df['Time T2'][i] = (df['Time T2'][i]/60).round(freq='S')

In [ ]:
df['test'] = (df['Swim time'] + df['Time T1'] + df['Run time'] + df['Cycle time'] + df['Time T2'] - df['Total time'])
zero = pd.to_timedelta('0 days 00:00:00')
df = df.fillna(zero)

In [ ]:
total_mean = df['Total time'].mean()
zero = pd.to_timedelta('0 days 00:00:00')
time = ['Swim time', 'Cycle time', 'Run time']
for col in time:
    prop = (df[col].mean()/total_mean)
    print(prop)
    for i in range(len(df)):
        if (df[col][i]) == zero:
            df[col][i] = (df['Total time'][i] * prop).round(freq='S')

In [ ]:
for i in range(len(df)):
    prove = df['Total time'][i] - (df['Swim time'][i] + df['Cycle time'][i] + df['Run time'][i])
    if prove != zero:
        df['Time T1'][i] = df['Time T2'][i] = (prove/2).round(freq='S')

In [ ]:
df['test'] = df['Total time'] - (df['Swim time'] + df['Time T1'] + df['Run time'] + df['Cycle time'] + df['Time T2'])
zero = pd.to_timedelta('0 days 00:00:00')
correct = df[df['test'] == zero]
incorrect = df[df['test'] != zero]
incorrect

In [ ]:
df = df[df['Year'] != 2015]
df = df[df['Year'] != 2005]
df.sort_values('Time T1')


In [ ]:
 for i in range(len(df)):
        df['Run distance'][i] = 42.0

In [ ]:
df = df.drop('test' , axis = 'columns')

In [ ]:
df.head()
df.info()

In [ ]:
df.to_csv('Cleaned_NORSMAN', index = False)